In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report, roc_curve, confusion_matrix, roc_auc_score, auc, recall_score, precision_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from scipy.stats import logistic
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier, LassoCV, Lasso, ElasticNet, LassoLars
from sklearn.metrics import f1_score, accuracy_score, classification_report, roc_curve, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from itertools import tee, islice
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk
from nltk.corpus import stopwords
import string
import re
import pickle

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('float_format', '{:f}'.format)

In [3]:
news = pd.read_json('C:/Users/GD/Downloads/news-category-dataset/News_Category_Dataset_v2.json', lines=True)

In [4]:
news.shape

(200853, 6)

In [5]:
news.category[news.category=='THE WORLDPOST'] = 'WORLDPOST'
news.category[news.category=='GREEN'] = 'ENVIRONMENT'
news.category[news.category=='CULTURE & ARTS'] = 'ARTS'
news.category[news.category=='COMEDY'] = 'ENTERTAINMENT'
news.category[(news.category=='BLACK VOICES') | (news.category=='LATINO VOICES') | (news.category=='QUEER VOICES')] = 'VOICES'
news.category[news.category=='STYLE'] = 'STYLE & BEAUTY'
news.category[news.category=='ARTS & CULTURE'] = 'ARTS'
news.category[news.category=='COLLEGE'] = 'EDUCATION'
news.category[news.category=='SCIENCE'] = 'TECH'
news.category[news.category=='WEDDINGS'] = 'GOOD NEWS'
news.category[news.category=='TASTE'] = 'FOOD & DRINK'
news.category[(news.category=='PARENTING') | (news.category=='FIFTY')] = 'PARENTS'
news.category[news.category=='WORLD NEWS'] = 'WORLDPOST'

In [6]:
news.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [7]:
news.category.nunique()

26

In [8]:
news.authors.nunique()

27993

In [9]:
news.authors.value_counts()

                                                                                                                       36620
Lee Moran                                                                                                               2423
Ron Dicker                                                                                                              1913
Reuters, Reuters                                                                                                        1562
Ed Mazza                                                                                                                1322
Cole Delbyck                                                                                                            1140
Andy McDonald                                                                                                           1068
Julia Brucculieri                                                                                                       1059


In [10]:
news[news.authors == 'Suzy Strutner'].category.value_counts()

TRAVEL            474
FOOD & DRINK       93
STYLE & BEAUTY     38
HEALTHY LIVING     38
ENTERTAINMENT       2
GOOD NEWS           2
WORLDPOST           1
BUSINESS            1
WEIRD NEWS          1
Name: category, dtype: int64

In [7]:
def textClean(text):
    text = re.sub("[^a-zA-Z]", ' ',text).strip()
    text = nltk.word_tokenize(text.lower())
    stops = set(stopwords.words("english"))
    stops = list(stops) + ['amp']
    text = " ".join([lemmatizer.lemmatize(w) for w in text if w not in stops])
    return(text)

In [8]:
%%time
news['Clean_headline'] = news.headline.map(textClean)
news['Clean_description'] = news.short_description.map(textClean)

Wall time: 21min


In [ ]:
news.category.unique()

In [22]:
wo = (str(list(news[news.category == 'CRIME']['Clean_headline']))).split()
def ngrams(lst, n):
    tlst = lst
    while True:
        a, b = tee(tlst)
        l = tuple(islice(a, n))
        if len(l) == n:
            yield l
            next(b)
            tlst = b
        else:
            break

b = Counter(ngrams(wo, 1))
b.most_common()

[(('police',), 280),
 (('man',), 227),
 (('shooting',), 209),
 (('year',), 178),
 (("'man",), 176),
 (('allegedly',), 157),
 (('killed',), 156),
 (('old',), 153),
 (('cop',), 152),
 (('found',), 142),
 (('officer',), 142),
 (('accused',), 137),
 (('shot',), 133),
 (("say',",), 132),
 (('arrested',), 131),
 (('suspect',), 125),
 (('woman',), 110),
 (('dead',), 109),
 (('murder',), 109),
 (("'police",), 105),
 (("cop',",), 101),
 (('teen',), 96),
 (('death',), 95),
 (('charged',), 94),
 (('killing',), 93),
 (('new',), 88),
 (('school',), 83),
 (('kill',), 82),
 (('shooter',), 82),
 (('gun',), 80),
 (('say',), 75),
 (('car',), 73),
 (('child',), 71),
 (("police',",), 71),
 (('get',), 67),
 (('girl',), 65),
 (("'woman",), 64),
 (("shooting',",), 62),
 (('sex',), 62),
 (('shoot',), 61),
 (('missing',), 61),
 (('fire',), 61),
 (('video',), 60),
 (('face',), 57),
 (("'cop",), 57),
 (('prison',), 57),
 (('show',), 56),
 (("video',",), 55),
 (('dy',), 55),
 (('attack',), 55),
 (('state',), 54),

In [ ]:
wo = (str(list(news[news.category == 'ENTERTAINMENT']['Clean_headline']))).split()
def ngrams(lst, n):
    tlst = lst
    while True:
        a, b = tee(tlst)
        l = tuple(islice(a, n))
        if len(l) == n:
            yield l
            next(b)
            tlst = b
        else:
            break

b = Counter(ngrams(wo, 2))
b.most_common()

In [ ]:
news[news.Clean_headline.str.contains('box office')].category.value_counts()

In [9]:
y = news.category
train, test, y_train, y_test = train_test_split(news, y, test_size=0.2, random_state=0, stratify= y)

In [10]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df = 5)

tfidf_train = tfidf_vectorizer.fit_transform(train['Clean_headline'])
tfidf_test = tfidf_vectorizer.transform(test['Clean_headline'])

In [11]:
tfidf_train

<160682x32200 sparse matrix of type '<class 'numpy.float64'>'
	with 1216185 stored elements in Compressed Sparse Row format>

In [12]:
%%time
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score

LR = LogisticRegression('l1')
LR.fit(tfidf_train, y_train)
y_predlr=LR.predict(tfidf_test)
print('LR test: ',accuracy_score(y_test, y_predlr))
print('LR train:',accuracy_score(y_train, LR.predict(tfidf_train)))
print('KAPPA SCORE: ',cohen_kappa_score(y_test,y_predlr))

LR test:  0.6334171417191506
LR train: 0.688677014226858
KAPPA SCORE:  0.6020816814045105
Wall time: 8min 6s


In [13]:
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(tfidf_train, y_train)
y_pred = nb.predict(tfidf_test)
print('LR test: ',accuracy_score(y_test, y_pred))
print('LR train:',accuracy_score(y_train, nb.predict(tfidf_train)))
print('KAPPA SCORE: ',cohen_kappa_score(y_test,y_pred))

LR test:  0.5443976998332131
LR train: 0.603210067089033
KAPPA SCORE:  0.49234824059297977


In [30]:
result = pd.DataFrame({'Actual_Class': y_test, 'Predicted_Class': y_pred})
result['error_test'] = np.where((result['Actual_Class'] == result['Predicted_Class']), 1,0).astype('int64')

In [48]:
result.groupby(['Actual_Class','Predicted_Class'])[['error_test']]

In [56]:
result.loc[result['error_test'] == 0][:10]

,Actual_Class,Predicted_Class,error_test
74193,TECH,WELLNESS,0
164936,PARENTS,POLITICS,0
143863,ENVIRONMENT,POLITICS,0
29558,MEDIA,POLITICS,0
158555,STYLE & BEAUTY,ENTERTAINMENT,0
164022,PARENTS,WELLNESS,0
30,POLITICS,ENTERTAINMENT,0
99741,WOMEN,POLITICS,0
109438,PARENTS,POLITICS,0
101368,IMPACT,POLITICS,0


In [59]:
# frame[['price1','price2']].groupby(frame['color']).mean()
result[['Actual_Class','Predicted_Class']].groupby(result['error_test']).count()

,Actual_Class,Predicted_Class
error_test,,
0,18302,18302
1,21869,21869


In [69]:
pd.pivot_table(result, values=['error_test'] ,index=['Actual_Class', 'Predicted_Class'], aggfunc=np.sum)

error_test
Actual_Class   Predicted_Class            
ARTS           ARTS                     94
               BUSINESS                  0
               ENTERTAINMENT             0
               ENVIRONMENT               0
               FOOD & DRINK              0
               GOOD NEWS                 0
               HOME & LIVING             0
               PARENTS                   0
               POLITICS                  0
               SPORTS                    0
               STYLE & BEAUTY            0
               TECH                      0
               TRAVEL                    0
               VOICES                    0
               WELLNESS                  0
               WOMEN                     0
               WORLDPOST                 0
BUSINESS       BUSINESS                184
               CRIME                     0
               ENTERTAINMENT             0
               FOOD & DRINK              0
               HEALTHY LIVING            0
               HOME & LIVING             0
               MONEY                     0
               PARENTS                   0
               POLITICS                  0
               SPORTS                    0
               STYLE & BEAUTY            0
               TECH                      0
               TRAVEL                    0
               VOICES                    0
               WELLNESS                  0
               WORLDPOST                 0
CRIME          ARTS                      0
               CRIME                   132
               ENTERTAINMENT             0
               PARENTS                   0
               POLITICS                  0
               SPORTS                    0
               STYLE & BEAUTY            0
               TRAVEL                    0
               VOICES                    0
               WELLNESS                  0
               WORLDPOST                 0
DIVORCE        DIVORCE                 161
               ENTERTAINMENT             0
               FOOD & DRINK              0
               GOOD NEWS                 0
               PARENTS                   0
               POLITICS                  0
               SPORTS                    0
               STYLE & BEAUTY            0
               TRAVEL                    0
               VOICES                    0
               WELLNESS                  0
               WORLDPOST                 0
EDUCATION      BUSINESS                  0
               CRIME                     0
               EDUCATION                11
               ENTERTAINMENT             0
               FOOD & DRINK              0
               PARENTS                   0
               POLITICS                  0
               SPORTS                    0
               STYLE & BEAUTY            0
               TRAVEL                    0
               VOICES                    0
               WELLNESS                  0
               WORLDPOST                 0
ENTERTAINMENT  ARTS                      0
               BUSINESS                  0
               DIVORCE                   0
               ENTERTAINMENT          3355
               ENVIRONMENT               0
               FOOD & DRINK              0
               GOOD NEWS                 0
               HEALTHY LIVING            0
               PARENTS                   0
               POLITICS                  0
               RELIGION                  0
               SPORTS                    0
               STYLE & BEAUTY            0
               TRAVEL                    0
               VOICES                    0
               WELLNESS                  0
               WORLDPOST                 0
ENVIRONMENT    BUSINESS                  0
               CRIME                     0
               ENTERTAINMENT             0
               ENVIRONMENT              74
               FOOD & DRINK              0
               GOOD NEWS         

In [70]:
sorted(list(zip(LR.coef_[0] ,tfidf_vectorizer.get_feature_names())), reverse = True)

[(8.19069100361118, 'nighter'),
 (7.7088518036933715, 'artist'),
 (7.383966107040858, 'opera'),
 (6.928546606762711, 'ballet'),
 (6.8943172801240395, 'photography'),
 (6.591413105538914, 'art'),
 (6.5806407080869125, 'theatre'),
 (6.494541889393929, 'aisle'),
 (6.154469737268217, 'stage door'),
 (6.107050140283552, 'sculpture'),
 (5.847764252445885, 'portrait'),
 (5.7792132912334475, 'exhibition'),
 (5.713205238961261, 'photographer'),
 (5.708838296192185, 'photograph'),
 (5.663805848897245, 'first nighter'),
 (5.6559167647684685, 'shakespeare'),
 (5.643266151881466, 'painting'),
 (5.58504183883241, 'theater'),
 (5.262756333266389, 'english'),
 (5.055101952491299, 'artwork'),
 (4.862366946695791, 'digital age'),
 (4.660631188126666, 'manhattan'),
 (4.591221274196278, 'wilson'),
 (4.45972331196371, 'new york'),
 (4.423564981125444, 'plastic'),
 (4.402200506504254, 'broadway'),
 (4.314732776318378, 'rock roll'),
 (4.239381047442543, 'mostly'),
 (4.2287354414084355, 'gallery'),
 (4.167372

In [71]:
sorted(list(zip(LR.coef_[0] ,tfidf_vectorizer.get_feature_names())))

[(-12.472417828511215, 'trump'),
 (-4.303740137310628, 'photo'),
 (-4.016284260424603, 'wedding'),
 (-3.0581562666992337, 'travel'),
 (-3.0147206276392797, 'week'),
 (-2.99192023359214, 'health'),
 (-2.9448607376252105, 'say'),
 (-2.5803844246760534, 'food'),
 (-2.5667418174771353, 'mom'),
 (-2.496244891566508, 'obama'),
 (-2.370852533073647, 'million'),
 (-2.306761943916968, 'clinton'),
 (-2.265443533390229, 'find'),
 (-2.2477030743594373, 'kid'),
 (-2.2336063293429786, 'attack'),
 (-2.1668016714338316, 'baby'),
 (-2.066977344205858, 'help'),
 (-2.06228963892218, 'police'),
 (-1.9609596048564157, 'report'),
 (-1.8025392979815988, 'gop'),
 (-1.7995282282223477, 'gay'),
 (-1.7635244858147108, 'cancer'),
 (-1.7272514227277536, 'tip'),
 (-1.7112192102913475, 'study'),
 (-1.6938344145395299, 'dog'),
 (-1.6811941373235435, 'recipe'),
 (-1.5850760585236932, 'people'),
 (-1.3924600767195174, 'day'),
 (-1.3686698940331683, 'parent'),
 (-1.3237279567818419, 'sleep'),
 (-1.2305819510195106, 'ele

In [15]:
pd.DataFrame(confusion_matrix(y_test, y_pred), columns = sorted(news.category.unique()), index = sorted(news.category.unique()))

,ARTS,BUSINESS,CRIME,DIVORCE,EDUCATION,ENTERTAINMENT,ENVIRONMENT,FOOD & DRINK,GOOD NEWS,HEALTHY LIVING,HOME & LIVING,IMPACT,MEDIA,MONEY,PARENTS,POLITICS,RELIGION,SPORTS,STYLE & BEAUTY,TECH,TRAVEL,VOICES,WEIRD NEWS,WELLNESS,WOMEN,WORLDPOST
ARTS,94,1,0,0,0,262,1,8,1,0,3,0,0,0,53,135,0,2,30,1,53,56,0,69,2,5
BUSINESS,0,184,2,0,0,72,0,16,0,2,1,0,0,1,43,614,0,3,14,7,21,10,0,188,0,9
CRIME,1,0,132,0,0,73,0,0,0,0,0,0,0,0,51,321,0,3,3,0,10,41,0,26,0,20
DIVORCE,0,0,0,161,0,132,0,4,10,0,0,0,0,0,128,107,0,4,16,0,3,18,0,100,0,2
EDUCATION,0,2,4,0,11,25,0,1,0,0,0,0,0,0,61,244,0,2,2,0,5,14,0,58,0,1
ENTERTAINMENT,1,3,0,1,0,3355,1,20,2,1,0,0,0,0,97,456,1,6,96,0,22,75,0,102,0,8
ENVIRONMENT,0,2,1,0,0,83,74,9,6,0,2,0,0,0,30,361,0,1,10,4,72,3,0,115,0,16
FOOD & DRINK,0,1,0,0,0,119,0,1119,0,0,6,0,0,0,45,82,0,0,28,1,70,4,0,188,0,1
GOOD NEWS,0,3,0,4,0,186,4,19,330,0,0,0,1,0,118,109,0,7,62,1,27,29,0,108,0,2
HEALTHY LIVING,0,2,0,0,0,63,0,24,0,18,2,1,0,0,96,222,0,1,11,1,11,7,0,872,0,8


In [21]:
y_test

56289           POLITICS
7634           WORLDPOST
147146            VOICES
80111             SPORTS
74193               TECH
25231           POLITICS
164936           PARENTS
46124           POLITICS
143863       ENVIRONMENT
29558              MEDIA
158555    STYLE & BEAUTY
90728            PARENTS
164022           PARENTS
30              POLITICS
82458            PARENTS
99741              WOMEN
109438           PARENTS
101368            IMPACT
89490          WORLDPOST
58852          GOOD NEWS
2696               MEDIA
16926           POLITICS
11245          WORLDPOST
4350              VOICES
35304             SPORTS
26454           POLITICS
2360              VOICES
151386          WELLNESS
151434          WELLNESS
114763          BUSINESS
161431          WELLNESS
67762               ARTS
71460             SPORTS
74313               ARTS
149354          WELLNESS
78273             VOICES
66086      ENTERTAINMENT
10299        ENVIRONMENT
17018          EDUCATION
184306           PARENTS


In [21]:
columns = ['box office', 'donald trump']

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df = 20, vocabulary = columns)

tfidf_train = tfidf_vectorizer.fit_transform(train['Clean_headline'])
tfidf_test = tfidf_vectorizer.transform(test['Clean_headline'])

LR = LogisticRegression()
LR.fit(tfidf_train, y_train)
y_pred_lr = LR.predict(tfidf_test)
print('LR test: ',accuracy_score(y_test, y_pred_lr))
print('LR train:',accuracy_score(y_train, LR.predict(tfidf_train)))

NameError: name 'train' is not defined

In [96]:
tfidf_train

<160682x2 sparse matrix of type '<class 'numpy.float64'>'
	with 3844 stored elements in Compressed Sparse Row format>

In [14]:
pd.DataFrame(confusion_matrix(y_test, y_pred_lr), columns = sorted(news.category.unique()), index = sorted(news.category.unique()))

NameError: name 'y_pred_lr' is not defined